In [ ]:
# Importing necessary libraries

#The cv2 module is the main module in OpenCV that provides developers with an easy-to-use interface for working with image and video processing functions
# it is used for frame processing
import cv2

# NumPy is a library for the Python programming language, adding support for large, multi-dimensional arrays and matrices, along with a large collection of high-level mathematical functions to operate on these arrays
import numpy as np

# It's a landmark's facial detector with pre-trained models, the dlib is used to estimate the location of 68 coordinates (x, y) that map the facial points on a person's face
import dlib

# A series of convenience functions to make basic image processing functions such as translation, rotation, resizing, skeletonization, displaying Matplotlib images, sorting contours, detecting edges, and much more easier with OpenCV and both Python 2.7 and Python 3.
from imutils import face_utils


from playsound import playsound


In [ ]:
# !pip install sounddevice


In [ ]:
import sounddevice as sd
import numpy as np
import threading
import time

In [ ]:
# Initialize the video capture object (0 represents the default webcam)
cap = cv2.VideoCapture(0)

# Initialize the face detector and facial landmarks predictor from dlib
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

# Define status variables for the blink detection
sleep = 0
drowsy = 0
active = 0
status = ""
color = (0, 0, 0)

# Define functions for computing distances and checking blinks
# It is calculating the distance between 2 landmarks 
def compute(ptA, ptB):
    dist = np.linalg.norm(ptA - ptB)
    return dist

# a and f are long distances, bd and ce are the short distances 
# In this we are calculating the ratio which is used for checking whether eyes are closed or not
# Here we are calculating the EAR (Eye Aspect Ratio)
def blinked(a, b, c, d, e, f):
    up = compute(b, d) + compute(c, e)
    down = compute(a, f)
    ratio = up / (2.0 * down)
    
    # Check for blinks based on ratio
    if ratio > 0.25:
        return 2
    elif 0.21 <= ratio <= 0.25:
        return 1
    else:
        return 0

# Define a function to perform OCR on a frame and print the extracted text
# def perform_ocr(frame):
#     # Convert the frame to grayscale for better OCR accuracy
#     gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

#     # Apply thresholding (binarization)
#     _, binary_frame = cv2.threshold(gray, 150, 255, cv2.THRESH_BINARY)

#     # Use Tesseract to extract text from the preprocessed frame
#     extracted_text = pytesseract.image_to_string(binary_frame, lang='eng')

#     # Perform OCR using pytesseract on the grayscale frame
#     # text = pytesseract.image_to_string(gray)

#     # Print the extracted text in the console
#     print("Extracted Text:", extracted_text)

# Continuously read frames from the video capture object


# Play alarm sound when sleeping state is detected
# def play_alarm():
#     # unique_alias = str(uuid.uuid4())
#     playsound("alarm.wav", block=False)
#     return 

def play_alarm():
    def alarm_sound():
        duration = 2  # Set the duration of the alarm sound
        fs = 44100  # Set the sampling frequency
        t = np.linspace(0, duration, int(fs * duration), False)  # Generate time vector
        data = np.sin(2 * np.pi * 440 * t)  # Generate sine wave data
        sd.play(data, samplerate=fs, blocking=True)  # Play the sound

    # Start a separate thread to play the alarm sound
    alarm_thread = threading.Thread(target=alarm_sound)
    alarm_thread.start()


while True:
    ret, frame = cap.read()
    
    # Check if the frame is read successfully
    if not ret:
        print("Error: Failed to read frame.")
        break

    # Convert the frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces in the grayscale frame
    faces = detector(gray)
    face_frame = frame.copy()
    
    # Process each detected face
    for face in faces:
        # It is used for finding the corner coordinates
        x1 = face.left()
        y1 = face.top()
        x2 = face.right()
        y2 = face.bottom()

        # Draw rectangles around detected faces
        cv2.rectangle(face_frame, (x1, y1), (x2, y2), (0, 255, 0), 2)

        # Get facial landmarks
        landmarks = predictor(gray, face)
        landmarks = face_utils.shape_to_np(landmarks)

        # Process eye blinks
        left_blink = blinked(landmarks[36], landmarks[37], landmarks[38], landmarks[41], landmarks[40], landmarks[39])
        right_blink = blinked(landmarks[42], landmarks[43], landmarks[44], landmarks[47], landmarks[46], landmarks[45])

        # Judge the eye blinks
        if left_blink == 0 or right_blink == 0:
            sleep += 1
            drowsy = 0
            active = 0
            if sleep > 6:
                status = "--SLEEPING--"
                color = (255, 0, 0)
                
                # Play alarm sound when the status is "SLEEPING"
                # playsound("alarm.wav")
                play_alarm()

        elif left_blink == 1 or right_blink == 1:
            sleep = 0
            active = 0
            drowsy += 1
            if drowsy > 6:
                status = "-Drowsy-"
                color = (0, 0, 255)
        else:
            drowsy = 0
            sleep = 0
            active += 1
            if active > 6:
                status = "Active"
                color = (0, 255, 0)
        
        # Display status on the frame
        cv2.putText(frame, status, (100, 100), cv2.FONT_HERSHEY_SIMPLEX, 1.2, color, 3)

        # Draw facial landmarks on the face frame
        for n in range(0, 68):
            (x, y) = landmarks[n]
            cv2.circle(face_frame, (x, y), 1, (255, 255, 255), -1)
    
    # Perform OCR and print the extracted text
    # perform_ocr(frame)

    # Show the original frame
    cv2.imshow("Frame", frame)

    # Show the frame with detected faces and landmarks
    if face_frame is not None:
        cv2

    cv2.imshow("Face Frame", face_frame)

    # Check for 'q' key to exit the loop
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture object and close OpenCV windows
cap.release()
cv2.destroyAllWindows()
